In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [2]:
!pip install --upgrade pyarrow

# **Importing necessary libraries**

---



---



In [3]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from datasets import load_dataset

# **Importing Dataset From Hugging face**

---



---



In [4]:
Dataset=load_dataset('mattmdjaga/human_parsing_dataset')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/17706 [00:00<?, ? examples/s]

# **Normalizing and Reshaping Dataset**

---



---



In [33]:
def reshape_normalize(image):
  X_train=[]
  Y_train=[]
  X_test=[]
  Y_test=[]
  new_shape=(256,400)
  for i in range(50):
    reshaped=np.array(image[i]['image'])/255.0
    Yreshaped=np.array(image[i]['mask'])/17.0
    X_train.append(cv.resize(reshaped,new_shape))
    Y_train.append(cv.resize(Yreshaped,new_shape))
  for i in range(50,70):
    reshaped=np.array(image[i]['image'])/255.0
    Ytreshaped=np.array(image[i]['mask'])/17.0
    X_test.append(cv.resize(reshaped,new_shape))
    Y_test.append(cv.resize(Ytreshaped,new_shape))

  return np.array(X_train),np.array(Y_train),np.array(X_test),np.array(Y_test)


# **Spliting Dataset to Train/Test images**

---



---



In [34]:
X_train,Y_train,X_test,Y_test=reshape_normalize(Dataset['train'])

# **Defining model Architecture**


---


---
Trying To use Less Convolutional filters. to decrease High model complexity and System requirement


In [36]:
from tensorflow.keras import layers, models

def UNet(input_shape=(400,256, 3)):
    inputs = layers.Input(shape=input_shape)

    # Down-sampling path
    A1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    A1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(A1)
    M1 = layers.MaxPooling2D((2, 2))(A1)

    A2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(M1)
    A2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(A2)
    M2 = layers.MaxPooling2D((2, 2))(A2)

    A3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(M2)
    A3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(A3)
    M3 = layers.MaxPooling2D((2, 2))(A3)

    A4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(M3)
    A4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(A4)
    M4 = layers.MaxPooling2D((2, 2))(A4)


    A5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(M4)
    A5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(A5)


    T1 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(A5)
    C1 = layers.concatenate([T1, A4])
    A6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(C1)
    A6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(A6)

    T2 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(A6)
    C2 = layers.concatenate([T2, A3])
    A7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(C2)
    A7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(A7)

    T3 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(A7)
    C3 = layers.concatenate([T3, A2])
    A8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(C3)
    A8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(A8)

    T4 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(A8)
    C4 = layers.concatenate([T4, A1])
    A9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(C4)
    A9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(A9)

    output = layers.Conv2D(1, (1, 1), activation='sigmoid')(A9)

    model = models.Model(inputs=inputs, outputs=output)
    return model

In [37]:
model=UNet()

# **Compiling Model:**

---



---

Loss = Binary CrossEntropy

Optimizer = ADAM

In [38]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# **Model Architecture**

---



---
Total params: 9240257 (35.25 MB)

Trainable params: 9240257 (35.25 MB)

Non-trainable params: 0 (0.00 Byte)


In [42]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 400, 256, 3)]        0         []                            
                                                                                                  
 conv2d_57 (Conv2D)          (None, 400, 256, 32)         896       ['input_4[0][0]']             
                                                                                                  
 conv2d_58 (Conv2D)          (None, 400, 256, 32)         9248      ['conv2d_57[0][0]']           
                                                                                                  
 max_pooling2d_12 (MaxPooli  (None, 200, 128, 32)         0         ['conv2d_58[0][0]']           
 ng2D)                                                                                      

In [41]:
model.fit(X_train,Y_train,epochs=5,batch_size=1)

Epoch 1/5
50/50 [==============================] - 257s 5s/step - loss: 0.3557 - accuracy: 0.7820
Epoch 2/5
50/50 [==============================] - 266s 5s/step - loss: 0.3171 - accuracy: 0.7821
Epoch 3/5
50/50 [==============================] - 254s 5s/step - loss: 0.3078 - accuracy: 0.7821
Epoch 4/5
50/50 [==============================] - 243s 5s/step - loss: 0.2932 - accuracy: 0.7821
Epoch 5/5
50/50 [==============================] - 245s 5s/step - loss: 0.3076 - accuracy: 0.7821


# **Got 78% of Accuracy on Test dataset**

---



---



In [46]:
loss,accuracy=model.evaluate(X_test,Y_test)

1/1 [==============================] - 28s 28s/step - loss: 0.2833 - accuracy: 0.7772
